In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

import matplotlib.pyplot as plt
%matplotlib inline

from HelmholtzSolver3D import *
from Mesh import *

def computeBoundaryImpedance(boundarySolution, iSpeaker):
    zSum = 0.0
    for i in range(aBoundaryMarkers.size):
        if i >= iSpeaker:
            p = soundPressure(boundarySolution.k, boundarySolution.aPhi[i])
            F = p * boundarySolution.parent.aArea[i]
            v = boundarySolution.aV[i]
            zSum += F / v
    return zSum   
    
(aVertices,_,_) = readVertexArray('SimpleSpeaker')
(aTriangles, aBoundaryMarkers) = readTriangleArray('SimpleSpeaker')
# TetGen orients these so that outside has clock-wise winding, BEM
# code needs it the other way around.
flipOrientation(aTriangles)
# Permute the triangle array (and boundary markers) with a perumtation 
# that sorts the boundary markers.
# This then allows computing integrals over specific parts of the model
# simple summation of contiguous sections of the triangle array, e.g. 
# computing the mechanical impedance caused by the interior box volume.
permutation = np.argsort(aBoundaryMarkers)
aResultArray = np.empty_like(aTriangles)
for i in range(permutation.size):
    aResultArray[i,:] = aTriangles[permutation[i], :]
aTriangles = aResultArray
aResultArray = np.empty_like(aBoundaryMarkers)
for i in range(permutation.size):
    aResultArray[i] = aBoundaryMarkers[permutation[i]]
aBoundaryMarkers = aResultArray
aHistogram = np.bincount(aBoundaryMarkers)
assert aHistogram[0] == 0, "There must be no triangles with marker 0."
iSpeaker = aHistogram[1]
interiorPoints = np.array([[0.500, 0.500, 0.500], # in center of speaker interior
#                           [0.750, 0.500, 0.500], # upward of center
#                           [0.500, 0.750, 0.500], # to the right of center
                           ], dtype=np.float32)


solver = HelmholtzSolver3D(aVertices, aTriangles)

# Set boundary condition based on boundary marker-values
boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
# indices up to iSpeaker are part of the enclosure
boundaryCondition.alpha[:iSpeaker] = 0.0
boundaryCondition.beta[:iSpeaker]  = 1.0
boundaryCondition.f[:iSpeaker] = 0.0
# indices from iSpeaker to the end are part of the speaker-membrane.
boundaryCondition.alpha[iSpeaker:] = 0.0
boundaryCondition.beta[iSpeaker:]  = 1.0
boundaryCondition.f[iSpeaker:] = 1.0
        
boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

deltaF = 2**(1.0/24.0)
low = 16 # Hz
samples = 240
aFrequency = np.empty(samples, dtype=np.float32)
aSoundPressure = np.empty(samples, dtype=np.float32)
aMembraneImpedance = np.empty(samples, dtype=np.complex64)

for i in range(samples):
    frequency = low * deltaF**i
    print "f[{}] = {}Hz".format(i, frequency)
    aFrequency[i] = frequency
    k = frequencyToWavenumber(frequency)
    boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)
    interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
    p = SoundMagnitude(soundPressure(k, interiorPhi[0]))
    aSoundPressure[i] = p
    aMembraneImpedance[i] = computeBoundaryImpedance(boundarySolution, iSpeaker)
    
plt.semilogx(aFrequency, aSoundPressure)
plt.title('Interior Sound Pressure')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Pressure [dB]')
plt.show()
plt.loglog(aFrequency, np.abs(aMembraneImpedance))
plt.title('Membrane Impedance')
plt.xlabel('Frequency [Hz]')
plt.ylabel('|Z|')
plt.show()